In [ ]:
import json
import pandas as pd
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
# load the model

model_name = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

## Sentence containing

In [ ]:
# function to generate a sentence containing the given word

def generate_sentence(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # remove the prompt part from the generated text

    if prompt in generated_text:
        generated_text = generated_text.replace(prompt, "").strip()
    return generated_text

# load the dataset
with open('/content/sentence_containing.json', 'r', encoding='utf-8') as file:
  dataset = json.load(file)

# select a random sample of 50 examples from the dataset

random.seed(42)  # For reproducibility
sample_keys = random.sample(list(dataset['examples'].keys()), 50)

# evaluate the random sample

results = []
for idx, key in enumerate(sample_keys):
    item = dataset['examples'][key]
    prompt = item['input']
    word = item['metadata']['word']
    pos_tags = item['metadata']['word_pos_tags']
    template_id = item['metadata']['template_id']

    # to generate the sentence
    generated_sentence = generate_sentence(prompt)

    # check if the generated sentence contains the word
    score = 1 if word in generated_sentence else 0

    # prepare the result
    result = {
        "example_number": idx + 1,
        "input": prompt,
        "word": word,
        "word_pos_tags": pos_tags,
        "generated_sentence": generated_sentence,
        "template_id": template_id,
        "score": score
    }

    results.append(result)

# function to evaluate the model's output with expected data
def evaluate_output(results):
    total_score = sum([result["score"] for result in results])
    return total_score

# get scores
total_score = evaluate_output(results)

# create a DataFrame
df = pd.DataFrame(results)

# display the output
print(df[:10])

# print summary of the evaluation
print(f"\nTotal Score: {total_score}")

# save the DataFrame and evaluation scores to CSV files
df.to_csv('RUGPT3_eval_sentence_containing.csv', index=False)

   example_number                                              input  \
0               1  Составьте предложение, используя слово "злобно...   
1               2      Напишите предложение со словом "поступить":\n   
2               3          Напишите предложение со словом "слайд":\n   
3               4  Придумайте предложение, в котором будет слово ...   
4               5  Придумайте предложение, в котором будет слово ...   
5               6      Напишите предложение со словом "сохранить":\n   
6               7         Напишите предложение со словом "свиток":\n   
7               8   Напишите предложение со словом "возвращаться":\n   
8               9  Составьте предложение, используя слово "похвал...   
9              10  Составьте предложение, используя слово "против...   

              word word_pos_tags  \
0           злобно         [ADV]   
1        поступить        [VERB]   
2            слайд        [NOUN]   
3           стоить        [VERB]   
4  неблагоприятный         

## Sentence not containing

In [ ]:
# to generate a sentence containing the given word

def generate_sentence(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # remove the prompt part from the generated text

    if prompt in generated_text:
        generated_text = generated_text.replace(prompt, "").strip()
    return generated_text

# load the dataset
with open('/content/sentence_not_containing.json', 'r', encoding='utf-8') as file:
  dataset = json.load(file)

# select a random sample of 50 examples from the dataset

random.seed(42)  # For reproducibility
sample_keys = random.sample(list(dataset['examples'].keys()), 50)

# evaluate the random sample

results = []
for idx, key in enumerate(sample_keys):
    item = dataset['examples'][key]
    prompt = item['input']
    word = item['metadata']['word']
    pos_tags = item['metadata']['word_pos_tags']
    template_id = item['metadata']['template_id']

    # to generate the sentence
    generated_sentence = generate_sentence(prompt)

    # check if the generated sentence contains the word
    score = 0 if word in generated_sentence else 1

    # prepare the result
    result = {
        "example_number": idx + 1,
        "input": prompt,
        "word": word,
        "word_pos_tags": pos_tags,
        "generated_sentence": generated_sentence,
        "template_id": template_id,
        "score": score
    }

    results.append(result)

# to evaluate the model's output with expected data
def evaluate_output(results):
    total_score = sum([result["score"] for result in results])
    return total_score

# get scores
total_score = evaluate_output(results)

# create a DataFrame
df = pd.DataFrame(results)

# display the output
print(df[:10])

# print summary of the evaluation
print(f"\nTotal Score: {total_score}")

# save the DataFrame and evaluation scores to CSV files
df.to_csv('RUGPT3_eval_sentence_not_containing.csv', index=False)

   example_number                                              input  \
0               1  Составьте предложение, не используя слово "про...   
1               2           Напишите предложение без слова "жаль":\n   
2               3       Напишите предложение без слова "избежать":\n   
3               4  Придумайте предложение, в котором не будет сло...   
4               5  Придумайте предложение, в котором не будет сло...   
5               6         Напишите предложение без слова "только":\n   
6               7     Напишите предложение без слова "выигрывать":\n   
7               8           Напишите предложение без слова "ниже":\n   
8               9  Составьте предложение, не используя слово "гол...   
9              10  Составьте предложение, не используя слово "впу...   

         word word_pos_tags  \
0     прожить        [VERB]   
1        жаль         [ADV]   
2    избежать        [VERB]   
3      письмо        [NOUN]   
4       осень        [NOUN]   
5      только        

## Sentence ends with word

In [ ]:
# function to generate a sentence that ends with the given word

def generate_sentence(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # remove the prompt part from the generated text

    if prompt in generated_text:
        generated_text = generated_text.replace(prompt, "").strip()
    return generated_text

# load the dataset

with open('/content/ends_with_word.json', 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# select a random sample of 50 examples from the dataset

random.seed(42)  # for reproducibility
sample_keys = random.sample(list(dataset['examples'].keys()), 50)

# evaluate the random sample

results = []
for idx, key in enumerate(sample_keys):
    item = dataset['examples'][key]
    prompt = item['input']
    word = item['metadata']['word']
    pos_tags = item['metadata']['word_pos_tags']
    template_id = item['metadata']['template_id']

    # to generate the sentence

    generated_sentence = generate_sentence(prompt)

    # check if the generated sentence ends with the word

    score = 1 if generated_sentence.endswith(word) else 0

    # prepare the result

    result = {
        "example_number": idx + 1,
        "input": prompt,
        "word": word,
        "word_pos_tags": pos_tags,
        "generated_sentence": generated_sentence,
        "template_id": template_id,
        "score": score
    }

    results.append(result)

# to evaluate the model's output with expected data

def evaluate_output(results):
    total_score = sum([result["score"] for result in results])
    return total_score

# get scores
total_score = evaluate_output(results)

# create a DataFrame
df = pd.DataFrame(results)

# display the output
print(df[:10])

# print summary of the evaluation
print(f"\nTotal Score: {total_score}")

# save the DataFrame and evaluation scores to CSV files
df.to_csv('RUGPT3_eval_ends_with_word.csv', index=False)

    example_number                                              input  \
0                1  Напишите предложение, заканчивающееся словом "...   
1                2  Напишите предложение, которое оканчивается на ...   
2                3  Напишите предложение, которое оканчивается на ...   
3                4  Напишите предложение, последнее слово которого...   
4                5  Напишите предложение, последнее слово которого...   
5                6  Напишите предложение, которое оканчивается на ...   
6                7  Напишите предложение, которое оканчивается на ...   
7                8  Напишите предложение, которое оканчивается на ...   
8                9  Напишите предложение, заканчивающееся словом "...   
9               10  Напишите предложение, заканчивающееся словом "...   
10              11  Напишите предложение, которое оканчивается на ...   
11              12  Напишите предложение, заканчивающееся словом "...   
12              13  Напишите предложение, последнее

## Sentence starts with Word

In [ ]:
# to generate a sentence that starts with the given word

def generate_sentence(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt part from the generated text

    if prompt in generated_text:
        generated_text = generated_text.replace(prompt, "").strip()
    return generated_text

# load the dataset

with open('/content/starts_with_word.json', 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# select a random sample of 50 examples from the dataset

random.seed(42)  # for reproducibility
sample_keys = random.sample(list(dataset['examples'].keys()), 50)

# evaluate the random sample

results = []
for idx, key in enumerate(sample_keys):
    item = dataset['examples'][key]
    prompt = item['input']
    word = item['metadata']['word']
    pos_tags = item['metadata']['word_pos_tags']
    template_id = item['metadata']['template_id']

    # to generate the sentence

    generated_sentence = generate_sentence(prompt)

    # check if the generated sentence starts with the word

    score = 1 if generated_sentence.startswith(word) else 0

    # prepare the result

    result = {
        "example_number": idx + 1,
        "input": prompt,
        "word": word,
        "word_pos_tags": pos_tags,
        "generated_sentence": generated_sentence,
        "template_id": template_id,
        "score": score
    }

    results.append(result)

# function to evaluate the model's output with expected data

def evaluate_output(results):
    total_score = sum([result["score"] for result in results])
    return total_score

# get scores
total_score = evaluate_output(results)

# create a DataFrame
df = pd.DataFrame(results)

# display the output
print(df[:10])

# print summary of the evaluation
print(f"\nTotal Score: {total_score}")

# save the DataFrame and evaluation scores to CSV files
df.to_csv('RUGPT3_eval_starts_with_word.csv', index=False)

    example_number                                              input  \
0                1  Напишите предложение, начинающееся на слово "ч...   
1                2  Напишите предложение, которое начинается на сл...   
2                3  Напишите предложение, которое начинается на сл...   
3                4  Напишите предложение, первое слово которого - ...   
4                5  Напишите предложение, первое слово которого - ...   
5                6  Напишите предложение, которое начинается на сл...   
6                7  Напишите предложение, которое начинается на сл...   
7                8  Напишите предложение, которое начинается на сл...   
8                9  Напишите предложение, начинающееся на слово "и...   
9               10  Напишите предложение, начинающееся на слово "о...   
10              11  Напишите предложение, которое начинается на сл...   
11              12  Напишите предложение, начинающееся на слово "б...   
12              13  Напишите предложение, первое сл